In [1]:
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [2]:
@everywhere using Distances

In [3]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

atis_data = open("atis_data.jsz","r") do fh
    deserialize(fh)    
end

@assert START_MARKER in atis_data["indexed_words"]
@assert START_MARKER in keys(atis_data["word_indexes"])
@assert END_MARKER in atis_data["indexed_words"]
@assert END_MARKER in keys(atis_data["word_indexes"])

In [4]:
# τ = Pheromone
# η = Prior
# α = pheremone weight
# β = prior weight
# ρ = pheremone evaporation
function get_edge_probs(τ::AbstractMatrix,η::AbstractMatrix, α, β)
    edges = τ.^α .* η.^β
    edges./=sum(edges,1) #Normalise to probabilities
    @inbounds edges[isnan(edges)]=0.0 #The terminal node will have a sum of zero
    edges
end
    
function get_edge_probs(τ::AbstractSparseMatrix,η::AbstractSparseMatrix, α, β)
    edges = τ.^α .* η.^β
    sums = sum(edges,1)
    
    I,J,V = findnz(edges)
    for idx in 1:length(V)
        @inbounds V[idx] /= sums[J[idx]]
    end
    sparse(I,J,V)
end

get_edge_probs (generic function with 2 methods)

In [5]:
?methodswith

search: 

```
methodswith(typ[, module or function][, showparents])
```

Return an array of methods with an argument of type `typ`. If optional `showparents` is `true`, also return arguments with a parent type of `typ`, excluding type `Any`.

The optional second argument restricts the search to a particular module or function.


In [6]:
@everywhere @inline function select_node{F<:Number}(edge_probs::AbstractVector{F})
    ys, _, probs = findnz(edge_probs'')
    select_node(zip(ys,probs))
end

@everywhere @inline function select_node{A, F<:Number}(edge_probs::Dict{A, F})
    select_node(collect(edge_probs))
end

@everywhere function select_node(edge_probs)
    cutoff = rand()
    total = 0.0
    @assert(length(edge_probs)>0, "No options from point $x")
    for (y,prob) in edge_probs
        total+=prob
        if total>=cutoff
            return y
        end
    end
    
    @assert(total>=cutoff, "x=$x, total= $total, cutoff=$cutoff") #Should Never Reach this point
end


methodswith



In [7]:
@everywhere function run_ant(edge_probs, start_index, end_index)
    path::Vector{Int} = [start_index] #1 is always the start node. Attached to it are unigram probs
    while (path[end]!=end_index)
        push!(path, select_node(edge_probs[:,path[end]]))
    end
    path    
end



function optimise(fit_fun, η, α, β,ρ, n_ants, n_gens, start_index, end_index)
    assert(α>=1.0)
    assert(β>=1.0)
    assert(0.0<=ρ<1.0)
    
    max_fit::Float64 = -Inf;
    fittest_path::Vector{Int} = []
    τ = ceil(η) #Since we take the element wise product with η, there is no point setting any elements that are zero in η
    
    for gen_ii in 1:n_gens
        if gen_ii%10==1 || gen_ii<10
            print("Gen: $gen_ii - fit: $max_fit\n")
        end
        
        edge_probs = get_edge_probs(τ,η, α, β)
        #edge_probs = SharedArray(Float64, size(τ))
        #edge_probs[:]=get_edge_probs(τ,η, α, β)
        
        
        path_fits::Vector{Tuple{Vector{Int},Float64}} = map(1:n_ants) do ant_id
            path::Vector{Int} = run_ant(edge_probs, start_index, end_index)
            fit::Float64 = fit_fun(path)
            (path, fit)
        end
        
    
        τ.*=(1.0-ρ) #Evaporation
        for (path, fit) in path_fits
            if fit>max_fit
                max_fit = fit
                fittest_path = path
                
                if fit==Inf #Perfect Fit
                    return fittest_path, max_fit
                end
            end

            
            @simd for node_ii in 1:length(path)-1
                @inbounds node = path[node_ii]
                @inbounds next_node = path[node_ii+1]
                @inbounds τ[next_node, node] += fit
            end
        end
    end #Next Generation
    
    fittest_path, max_fit
end

optimise (generic function with 1 method)

In [8]:
function lookup_sowe(data, sent)
    sum([data["LL"][:,data["word_indexes"][word]] for word in split(sent)]) 
end

lookup_sowe (generic function with 1 method)

In [9]:
function optimise(data::Dict, target_sent::AbstractString, α, β, ρ, n_ants, n_gens)
    target::Vector{Float64} = lookup_sowe(data,target_sent)
    optimise(data, target, α, β,ρ, n_ants, n_gens)
end

@inline function optimise(data::Dict, target::AbstractVector, α, β,ρ, n_ants, n_gens)
    #rLLs = Dict([id=>put!(RemoteRef(), data["LL"]) for id in workers()])
    #LL = SharedArray(Float64, size(atis_data["LL"]))
    LL =atis_data["LL"]
    @inline function fitness(path)
        #Fitter is larger
        @inbounds path_end = sum([LL[:,ii] for ii in path]) 
        if all(path_end.==0.0)
            0.0 #Cosine dist from origin to anywhere is zero.
            #This occurs for forced stopwords
        else
            (1.0/cosine_dist(path_end, target))/length(path)
        end
    end
    
    start_index = data["word_indexes"][START_MARKER]
    end_index = data["word_indexes"][END_MARKER]
    
    
    #return fitness, data["bigrams"], α, β,ρ, n_ants, n_gens, start_index, end_index
    path, score = optimise(fitness, data["bigrams"], α, β,ρ, n_ants, n_gens, start_index, end_index)
    score,[data["indexed_words"][ii] for ii in path]
end

optimise (generic function with 3 methods)

In [10]:
function subset_splits(xs)
    ss = subsets(xs) |> collect
    xs = zip(ss,reverse(ss)) |> collect    
    xs[1:div(end,2)]
end

function nonempty_subset_splits(xs)
    subset_splits(xs)[2:end]
end


nonempty_subset_splits (generic function with 1 method)

In [11]:
nonempty_subset_splits([1])

0-element Array{Tuple{Array{Int64,1},Array{Int64,1}},1}

In [12]:
function get_prior_fun(uni_occur, bi_joint, prob_length, end_index)
    n_edges=length(uni_occur)
    bi_marginal = sum(bi_joint,1)
    bi_conditional = bi_joint./bi_marginal
    


    
    function joint(aa)
        bi_marginal[aa]
    end
    
    function joint(aa,bb)
        bi_joint[aa, bb]
    end
    
    joint_cache = Dict{Set{Int64}, Float64}()
    function joint(xxs...) 
        @assert(length(xxs)>=3)
        xs_set = Set(xxs)
        if !(haskey(joint_cache,xs_set))
            joint_cache[xs_set] = mean(Float64[(joint(aas...)*joint(bbs...)) for (aas,bbs) in  nonempty_subset_splits(xxs)])
        end
        joint_cache[xs_set]
        
    end
    
    function conditional_occurance(ws::Vector{Int64})
        if length(ws) == 0
            uni_occur
        elseif length(ws)==1
            bi_conditional[:,ws[1]]
        else
            #Float64[joint(aa,ws...)./joint(ws...) for aa in 1:n_edges]
            
            [joint(ws) for aa in 1:n_edges]
        end
    end
    
    function prior(ws)
        #if  length(ws) > length(prob_length) #If it is longer than the tail of length distro, just force temrination
        #    force_end_pmf = zeros(n_edges) #This PMF forces the sentence to end, if it is returned
        #    force_end_pmf[end_index]=1.0
        #    force_end_pmf
        #else 
        #    #Either do not end (P=1-P_end) or end (P=P_end)
        #    prob_end = prob_length[length(ws)]   #Chance to end at this length
             prob_occurance = conditional_occurance(ws)
            
        #    choices_prob = (1.0-prob_end).*prob_occurance
        #    choices_prob[end_index]=prob_end
        #    choices_prob
        #end 
    end
    
end

function get_prior_fun(data::Dict)
    get_prior_fun(data["unioccur"],data["bioccur"], data["length_prob"], data["word_indexes"][END_MARKER])
end

get_prior_fun (generic function with 2 methods)

In [13]:
sum(atis_data["bioccur"],1)[1,2]

0.004786453537198461

In [ ]:
atis_prior = get_prior_fun(atis_data)

In [ ]:
atis_prior(Int64[1,4])

In [ ]:
maximum( atis_data["bioccur"])

In [ ]:
sum(bo[6,:])

In [ ]:
bo[:,6]./uo

In [ ]:


nonempty_subset_splits([1,2,3,4]) |> collect


In [ ]:
?zip!


In [ ]:
@time optimise(atis_data,"what is the longest flight", 1, 1, 0.3, 10000,20)
#code_warntype(optimise, map(typeof,params))

In [ ]:
optimise(atis_data,"what is the shortest flight", 1, 1, 0.1, 10000,20)

In [ ]:
optimise(atis_data,"what is the first flight from washington to washington", 1, 1, 0.1, 10000,20)

In [ ]:
optimise(atis_data,"what are the ground transport options", 1, 1, 0.1, 1000,20)

In [ ]:
optimise(atis_data,"where can i go", 1, 1, 0.1, 1000,20)

In [ ]:
atis_data["word_indexes"][START_MARKER]
    

In [ ]:
`git push""` |> run

In [ ]:
`git commit -m="Trioccuance is in" -a` |> run